In [1]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("./데이터자료/9-1. MinMax 데이터셋.csv")
feature = pd.read_csv("./데이터자료/8-1. MinMAx feature.csv")
feature 


,mda_feature,logit_feature,lasso_feature,ela_feature,same_feature
0,자기자본구성비율,매출액증가율,자기자본증가율,자기자본증가율,자기자본배율
1,매출액순이익률,유형자산증가율,총자본증가율,총자본증가율,자기자본구성비율
2,순운전자본비율,재고자산증가율,매출액순이익률,매출액순이익률,매출액순이익률
3,순운전자본회전률,비유동자산증가율,자기자본순이익률,자기자본순이익률,총자본증가율
4,총자본투자효율,유동자산증가율,순운전자본비율,총자본사업이익률,매출채권회전기간
5,자기자본순이익률,현금비율,유동자산구성비율,총자본순이익률,순운전자본비율
6,총자본증가율,이자보상배율이자비용,자기자본배율,순운전자본비율,순운전자본회전률
7,총자본사업이익률,비유동비율,매출채권회전률,자기자본구성비율,자기자본순이익률
8,기계투자효율,자기자본구성비율,매입채무회전기간,자기자본배율,총자본투자효율
9,1주당순이익,자기자본배율,매출채권회전기간,매출채권회전기간,매출채권회전률


In [3]:
train = df[df["회계년"] <= 2018].reset_index(drop='index')
test  = df[df["회계년"] > 2018].reset_index(drop='index')

In [4]:
x_train = train.drop("label",axis=1)
y_train = train["label"]
x_test =test.drop("label",axis=1)
y_test = test["label"]

In [5]:
mda_feature=feature["mda_feature"].tolist()
logit_feature=feature["logit_feature"].tolist()
lasso_feature=feature["lasso_feature"].tolist()
ela_feature=feature["ela_feature"].tolist()
same_feature=feature["same_feature"].tolist()

In [6]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

# 1. mda_feature

In [7]:
mda_x_train = x_train[mda_feature].reset_index(drop='index')
mda_x_test = x_test[mda_feature].reset_index(drop='index')
mda_y_train = train["label"]
mda_y_test = test["label"]

In [8]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(mda_x_train, mda_y_train)
lda_pred = lda_clf.predict(mda_x_test)
lda_pred_proba = lda_clf.predict_proba(mda_x_test)

In [9]:
print("lda") 
print(get_clf_eval(mda_y_test, lda_pred))

lda
오차행렬
[[2252   63]
 [  32   18]]
정확도: 0.9598, 정밀도 : 0.2222, 재현율:0.3600,F1 스코어:0.2748
ROC 스코어: 0.6664, PR score : 0.0935
None


In [10]:
# 디시전트리 구현
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)     
dt_clf.fit(mda_x_train, mda_y_train)
dt_pred = dt_clf.predict(mda_x_test)
dt_pred_proba = dt_clf.predict_proba(mda_x_test)                                    

In [11]:
print("디시전트리") 
print(get_clf_eval(mda_y_test, dt_pred))

디시전트리
오차행렬
[[2265   50]
 [  42    8]]
정확도: 0.9611, 정밀도 : 0.1379, 재현율:0.1600,F1 스코어:0.1481
ROC 스코어: 0.5692, PR score : 0.0398
None


In [12]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)     
rf_clf.fit(mda_x_train, mda_y_train)
rf_pred = rf_clf.predict(mda_x_test)
rf_pred_proba = rf_clf.predict_proba(mda_x_test)

In [13]:
print("랜덤포레스트") 
get_clf_eval(mda_y_test, rf_pred)

랜덤포레스트
오차행렬
[[2308    7]
 [  41    9]]
정확도: 0.9797, 정밀도 : 0.5625, 재현율:0.1800,F1 스코어:0.2727
ROC 스코어: 0.5885, PR score : 0.1186


In [14]:
# 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(mda_x_train, mda_y_train)
lr_pred = lr_clf.predict(mda_x_test)
lr_pred_proba = lr_clf.predict_proba(mda_x_test)

In [15]:
print("로지스틱") 
get_clf_eval(mda_y_test, lr_pred)

로지스틱
오차행렬
[[2310    5]
 [  50    0]]
정확도: 0.9767, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.4989, PR score : 0.0211


In [16]:
# KNN 모형 구현
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(mda_x_train, mda_y_train)
knn_pred = knn_clf.predict(mda_x_test)
knn_pred_proba = knn_clf.predict_proba(mda_x_test)

c:\Users\young\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [17]:
print("knn") 
get_clf_eval(mda_y_test, knn_pred)

knn
오차행렬
[[2312    3]
 [  42    8]]
정확도: 0.9810, 정밀도 : 0.7273, 재현율:0.1600,F1 스코어:0.2623
ROC 스코어: 0.5794, PR score : 0.1341


In [18]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 
mda_x_train2 = mda_x_train.copy()
mda_x_test2 = mda_x_test.copy()
mda_y_train2 = mda_y_train.copy()
mda_y_test2 = mda_y_test.copy()
mda_x_train2.columns = [mda_feature]
mda_x_test2.columns =[mda_feature]

xgb_wrapper = XGBClassifier()

xgb_wrapper.fit(mda_x_train2, mda_y_train2, verbose = True)


xgb_pred = xgb_wrapper.predict(mda_x_test)
xgb_pred_proba = xgb_wrapper.predict_proba(mda_x_test)

In [19]:
print("xgb") 
get_clf_eval(mda_y_test, xgb_pred)

xgb
오차행렬
[[2302   13]
 [  40   10]]
정확도: 0.9776, 정밀도 : 0.4348, 재현율:0.2000,F1 스코어:0.2740
ROC 스코어: 0.5972, PR score : 0.1039


# 2. lasso_feature

In [20]:
logit_x_train = x_train[logit_feature].reset_index(drop='index')
logit_x_test = x_test[logit_feature].reset_index(drop='index')
logit_y_train = train["label"]
logit_y_test = test["label"]

In [21]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(logit_x_train, logit_y_train)
lda_pred = lda_clf.predict(logit_x_test)
lda_pred_proba = lda_clf.predict_proba(logit_x_test)

In [22]:
print("lda") 
print(get_clf_eval(logit_y_test, lda_pred))

lda
오차행렬
[[2304   11]
 [  45    5]]
정확도: 0.9763, 정밀도 : 0.3125, 재현율:0.1000,F1 스코어:0.1515
ROC 스코어: 0.5476, PR score : 0.0503
None


In [23]:
# 디시전트리 구현
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)     
dt_clf.fit(logit_x_train, logit_y_train)
dt_pred = dt_clf.predict(logit_x_test)
dt_pred_proba = dt_clf.predict_proba(logit_x_test)                                    

In [24]:
print("디시전트리") 
print(get_clf_eval(logit_y_test, dt_pred))

디시전트리
오차행렬
[[2257   58]
 [  43    7]]
정확도: 0.9573, 정밀도 : 0.1077, 재현율:0.1400,F1 스코어:0.1217
ROC 스코어: 0.5575, PR score : 0.0333
None


In [25]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)     
rf_clf.fit(logit_x_train, logit_y_train)
rf_pred = rf_clf.predict(logit_x_test)
rf_pred_proba = rf_clf.predict_proba(logit_x_test)

In [26]:
print("랜덤포레스트") 
get_clf_eval(logit_y_test, rf_pred)

랜덤포레스트
오차행렬
[[2304   11]
 [  43    7]]
정확도: 0.9772, 정밀도 : 0.3889, 재현율:0.1400,F1 스코어:0.2059
ROC 스코어: 0.5676, PR score : 0.0726


In [27]:
# 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(logit_x_train, logit_y_train)
lr_pred = lr_clf.predict(logit_x_test)
lr_pred_proba = lr_clf.predict_proba(logit_x_test)

In [28]:
print("로지스틱") 
get_clf_eval(logit_y_test, lr_pred)

로지스틱
오차행렬
[[2314    1]
 [  49    1]]
정확도: 0.9789, 정밀도 : 0.5000, 재현율:0.0200,F1 스코어:0.0385
ROC 스코어: 0.5098, PR score : 0.0307


In [29]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(logit_x_train, logit_y_train)
knn_pred = knn_clf.predict(logit_x_test)
knn_pred_proba = knn_clf.predict_proba(logit_x_test)

c:\Users\young\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [30]:
print("knn") 
get_clf_eval(logit_y_test, knn_pred)

knn
오차행렬
[[2312    3]
 [  48    2]]
정확도: 0.9784, 정밀도 : 0.4000, 재현율:0.0400,F1 스코어:0.0727
ROC 스코어: 0.5194, PR score : 0.0363


In [31]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 
logit_x_train2 = logit_x_train.copy()
logit_x_test2 = logit_x_test.copy()
logit_y_train2 = logit_y_train.copy()
logit_y_test2 = logit_y_test.copy()
logit_x_train2.columns = [logit_feature]
logit_x_test2.columns =[logit_feature]

xgb_wrapper = XGBClassifier()

xgb_wrapper.fit(logit_x_train2, logit_y_train2, verbose = True)


xgb_pred = xgb_wrapper.predict(logit_x_test)
xgb_pred_proba = xgb_wrapper.predict_proba(logit_x_test)

In [32]:
print("xgb") 
get_clf_eval(logit_y_test, xgb_pred)

xgb
오차행렬
[[2301   14]
 [  43    7]]
정확도: 0.9759, 정밀도 : 0.3333, 재현율:0.1400,F1 스코어:0.1972
ROC 스코어: 0.5670, PR score : 0.0648


In [33]:
class Perceptron(object):
    
    def __init__(self, eta = 0.1, n_iter = 50, random_state = 1): #초기화
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
        
    
    def fit(self, X, y): #학습
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc = 0, scale = 0,
                             size = 1 + X.shape[1])
        
        self.errors_ = []
        for _ in range(self.n_iter):
            errors = 0
            for i in range(len(X)):
                xi = X.iloc[i].values
                target = y[i]
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def predict(self, X): #추론
        return np.where(self.net_input(X) >= 0.0, 1, 0)
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

pc_clf = Perceptron(n_iter = 10)
pc_clf.fit(logit_x_train, logit_y_train)
pc_pred = pc_clf.predict(logit_x_test)

In [34]:
print("퍼셉트론") 
get_clf_eval(logit_y_test, pc_pred)

퍼셉트론
오차행렬
[[2313    2]
 [  49    1]]
정확도: 0.9784, 정밀도 : 0.3333, 재현율:0.0200,F1 스코어:0.0377
ROC 스코어: 0.5096, PR score : 0.0274


# 3. lasso

In [35]:
lasso_x_train = x_train[lasso_feature].reset_index(drop='index')
lasso_x_test = x_test[lasso_feature].reset_index(drop='index')
lasso_y_train = train["label"]
lasso_y_test = test["label"]

In [36]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(lasso_x_train, lasso_y_train)
lda_pred = lda_clf.predict(lasso_x_test)
lda_pred_proba = lda_clf.predict_proba(lasso_x_test)

In [37]:
print("lda") 
print(get_clf_eval(lasso_y_test, lda_pred))

lda
오차행렬
[[2255   60]
 [  30   20]]
정확도: 0.9619, 정밀도 : 0.2500, 재현율:0.4000,F1 스코어:0.3077
ROC 스코어: 0.6870, PR score : 0.1127
None


In [38]:
# 디시전트리 구현
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)     
dt_clf.fit(lasso_x_train, lasso_y_train)
dt_pred = dt_clf.predict(lasso_x_test)
dt_pred_proba = dt_clf.predict_proba(lasso_x_test)                                    

In [39]:
print("디시전트리") 
print(get_clf_eval(lasso_y_test, dt_pred))

디시전트리
오차행렬
[[2269   46]
 [  40   10]]
정확도: 0.9636, 정밀도 : 0.1786, 재현율:0.2000,F1 스코어:0.1887
ROC 스코어: 0.5901, PR score : 0.0526
None


In [40]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)     
rf_clf.fit(lasso_x_train, lasso_y_train)
rf_pred = rf_clf.predict(lasso_x_test)
rf_pred_proba = rf_clf.predict_proba(lasso_x_test)

In [41]:
print("랜덤포레스트") 
get_clf_eval(lasso_y_test, rf_pred)

랜덤포레스트
오차행렬
[[2307    8]
 [  42    8]]
정확도: 0.9789, 정밀도 : 0.5000, 재현율:0.1600,F1 스코어:0.2424
ROC 스코어: 0.5783, PR score : 0.0978


In [42]:
# 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(lasso_x_train, lasso_y_train)
lr_pred = lr_clf.predict(lasso_x_test)
lr_pred_proba = lr_clf.predict_proba(lasso_x_test)

In [43]:
print("로지스틱") 
get_clf_eval(lasso_y_test, lr_pred)

로지스틱
오차행렬
[[2311    4]
 [  48    2]]
정확도: 0.9780, 정밀도 : 0.3333, 재현율:0.0400,F1 스코어:0.0714
ROC 스코어: 0.5191, PR score : 0.0336


In [44]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(lasso_x_train, lasso_y_train)
knn_pred = knn_clf.predict(lasso_x_test)
knn_pred_proba = knn_clf.predict_proba(lasso_x_test)

c:\Users\young\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [45]:
print("knn") 
get_clf_eval(lasso_y_test, knn_pred)

knn
오차행렬
[[2314    1]
 [  49    1]]
정확도: 0.9789, 정밀도 : 0.5000, 재현율:0.0200,F1 스코어:0.0385
ROC 스코어: 0.5098, PR score : 0.0307


In [46]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 
lasso_x_train2 = lasso_x_train.copy()
lasso_x_test2 = lasso_x_test.copy()
lasso_y_train2 = lasso_y_train.copy()
lasso_y_test2 = lasso_y_test.copy()
lasso_x_train2.columns = [lasso_feature]
lasso_x_test2.columns =[lasso_feature]

xgb_wrapper = XGBClassifier()

xgb_wrapper.fit(lasso_x_train2, lasso_y_train2, verbose = True)


xgb_pred = xgb_wrapper.predict(lasso_x_test)
xgb_pred_proba = xgb_wrapper.predict_proba(lasso_x_test)

In [47]:
print("xgb") 
get_clf_eval(lasso_y_test, xgb_pred)

xgb
오차행렬
[[2304   11]
 [  41    9]]
정확도: 0.9780, 정밀도 : 0.4500, 재현율:0.1800,F1 스코어:0.2571
ROC 스코어: 0.5876, PR score : 0.0983


In [48]:
class Perceptron(object):
    
    def __init__(self, eta = 0.1, n_iter = 50, random_state = 1): #초기화
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
        
    
    def fit(self, X, y): #학습
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc = 0, scale = 0,
                             size = 1 + X.shape[1])
        
        self.errors_ = []
        for _ in range(self.n_iter):
            errors = 0
            for i in range(len(X)):
                xi = X.iloc[i].values
                target = y[i]
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def predict(self, X): #추론
        return np.where(self.net_input(X) >= 0.0, 1, 0)
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

pc_clf = Perceptron(n_iter = 10)
pc_clf.fit(lasso_x_train, lasso_y_train)
pc_pred = pc_clf.predict(lasso_x_test)

In [49]:
print("퍼셉트론") 
get_clf_eval(lasso_y_test, pc_pred)

퍼셉트론
오차행렬
[[2308    7]
 [  47    3]]
정확도: 0.9772, 정밀도 : 0.3000, 재현율:0.0600,F1 스코어:0.1000
ROC 스코어: 0.5285, PR score : 0.0379


In [50]:
ela_x_train = x_train[ela_feature].reset_index(drop='index')
ela_x_test = x_test[ela_feature].reset_index(drop='index')
ela_y_train = train["label"]
ela_y_test = test["label"]

In [51]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(ela_x_train, ela_y_train)
lda_pred = lda_clf.predict(ela_x_test)
lda_pred_proba = lda_clf.predict_proba(ela_x_test)

In [52]:
print("lda") 
print(get_clf_eval(ela_y_test, lda_pred))

lda
오차행렬
[[2254   61]
 [  31   19]]
정확도: 0.9611, 정밀도 : 0.2375, 재현율:0.3800,F1 스코어:0.2923
ROC 스코어: 0.6768, PR score : 0.1034
None


In [53]:
# 디시전트리 구현
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)     
dt_clf.fit(ela_x_train, ela_y_train)
dt_pred = dt_clf.predict(ela_x_test)
dt_pred_proba = dt_clf.predict_proba(ela_x_test)                                    

In [54]:
print("디시전트리") 
print(get_clf_eval(ela_y_test, dt_pred))

디시전트리
오차행렬
[[2268   47]
 [  40   10]]
정확도: 0.9632, 정밀도 : 0.1754, 재현율:0.2000,F1 스코어:0.1869
ROC 스코어: 0.5898, PR score : 0.0520
None


In [55]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)     
rf_clf.fit(ela_x_train, ela_y_train)
rf_pred = rf_clf.predict(ela_x_test)
rf_pred_proba = rf_clf.predict_proba(ela_x_test)

In [56]:
print("랜덤포레스트") 
get_clf_eval(ela_y_test, rf_pred)

랜덤포레스트
오차행렬
[[2306    9]
 [  42    8]]
정확도: 0.9784, 정밀도 : 0.4706, 재현율:0.1600,F1 스코어:0.2388
ROC 스코어: 0.5781, PR score : 0.0931


In [57]:
# 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(ela_x_train, ela_y_train)
lr_pred = lr_clf.predict(ela_x_test)
lr_pred_proba = lr_clf.predict_proba(ela_x_test)

In [58]:
print("로지스틱") 
get_clf_eval(ela_y_test, lr_pred)

로지스틱
오차행렬
[[2311    4]
 [  48    2]]
정확도: 0.9780, 정밀도 : 0.3333, 재현율:0.0400,F1 스코어:0.0714
ROC 스코어: 0.5191, PR score : 0.0336


In [59]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(ela_x_train, ela_y_train)
knn_pred = knn_clf.predict(ela_x_test)
knn_pred_proba = knn_clf.predict_proba(ela_x_test)

c:\Users\young\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [60]:
print("knn") 
get_clf_eval(ela_y_test, knn_pred)

knn
오차행렬
[[2310    5]
 [  43    7]]
정확도: 0.9797, 정밀도 : 0.5833, 재현율:0.1400,F1 스코어:0.2258
ROC 스코어: 0.5689, PR score : 0.0998


In [61]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 
ela_x_train2 = ela_x_train.copy()
ela_x_test2 = ela_x_test.copy()
ela_y_train2 = ela_y_train.copy()
ela_y_test2 = ela_y_test.copy()
ela_x_train2.columns = [ela_feature]
ela_x_test2.columns =[ela_feature]

xgb_wrapper = XGBClassifier()

xgb_wrapper.fit(ela_x_train2, ela_y_train2, verbose = True)


xgb_pred = xgb_wrapper.predict(ela_x_test)
xgb_pred_proba = xgb_wrapper.predict_proba(ela_x_test)

In [62]:
print("xgb") 
get_clf_eval(ela_y_test, xgb_pred)

xgb
오차행렬
[[2300   15]
 [  39   11]]
정확도: 0.9772, 정밀도 : 0.4231, 재현율:0.2200,F1 스코어:0.2895
ROC 스코어: 0.6068, PR score : 0.1096


In [63]:
class Perceptron(object):
    
    def __init__(self, eta = 0.1, n_iter = 50, random_state = 1): #초기화
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
        
    
    def fit(self, X, y): #학습
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc = 0, scale = 0,
                             size = 1 + X.shape[1])
        
        self.errors_ = []
        for _ in range(self.n_iter):
            errors = 0
            for i in range(len(X)):
                xi = X.iloc[i].values
                target = y[i]
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def predict(self, X): #추론
        return np.where(self.net_input(X) >= 0.0, 1, 0)
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

pc_clf = Perceptron(n_iter = 10)
pc_clf.fit(ela_x_train, ela_y_train)
pc_pred = pc_clf.predict(ela_x_test)

In [64]:
print("퍼셉트론") 
get_clf_eval(ela_y_test, pc_pred)

퍼셉트론
오차행렬
[[2310    5]
 [  49    1]]
정확도: 0.9772, 정밀도 : 0.1667, 재현율:0.0200,F1 스코어:0.0357
ROC 스코어: 0.5089, PR score : 0.0241


# 5. 공통

In [65]:
same_x_train = x_train[same_feature].reset_index(drop='index')
same_x_test = x_test[same_feature].reset_index(drop='index')
same_y_train = train["label"]
same_y_test = test["label"]

In [66]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(same_x_train, same_y_train)
lda_pred = lda_clf.predict(same_x_test)
lda_pred_proba = lda_clf.predict_proba(same_x_test)

In [67]:
print("lda") 
print(get_clf_eval(same_y_test, lda_pred))

lda
오차행렬
[[2260   55]
 [  31   19]]
정확도: 0.9636, 정밀도 : 0.2568, 재현율:0.3800,F1 스코어:0.3065
ROC 스코어: 0.6781, PR score : 0.1107
None


In [68]:
# 디시전트리 구현
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)     
dt_clf.fit(same_x_train, same_y_train)
dt_pred = dt_clf.predict(same_x_test)
dt_pred_proba = dt_clf.predict_proba(same_x_test)                                    

In [69]:
print("디시전트리") 
print(get_clf_eval(same_y_test, dt_pred))

디시전트리
오차행렬
[[2274   41]
 [  39   11]]
정확도: 0.9662, 정밀도 : 0.2115, 재현율:0.2200,F1 스코어:0.2157
ROC 스코어: 0.6011, PR score : 0.0630
None


In [70]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)     
rf_clf.fit(same_x_train, same_y_train)
rf_pred = rf_clf.predict(same_x_test)
rf_pred_proba = rf_clf.predict_proba(same_x_test)

In [71]:
print("랜덤포레스트") 
get_clf_eval(same_y_test, rf_pred)

랜덤포레스트
오차행렬
[[2306    9]
 [  42    8]]
정확도: 0.9784, 정밀도 : 0.4706, 재현율:0.1600,F1 스코어:0.2388
ROC 스코어: 0.5781, PR score : 0.0931


In [72]:
# 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(same_x_train, same_y_train)
lr_pred = lr_clf.predict(same_x_test)
lr_pred_proba = lr_clf.predict_proba(same_x_test)

In [73]:
print("로지스틱") 
get_clf_eval(same_y_test, lr_pred)

로지스틱
오차행렬
[[2310    5]
 [  48    2]]
정확도: 0.9776, 정밀도 : 0.2857, 재현율:0.0400,F1 스코어:0.0702
ROC 스코어: 0.5189, PR score : 0.0317


In [74]:
# KNN 모형 구현
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(same_x_train, same_y_train)
knn_pred = knn_clf.predict(same_x_test)
knn_pred_proba = knn_clf.predict_proba(same_x_test)

c:\Users\young\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [75]:
print("knn") 
get_clf_eval(same_y_test, knn_pred)

knn
오차행렬
[[2311    4]
 [  44    6]]
정확도: 0.9797, 정밀도 : 0.6000, 재현율:0.1200,F1 스코어:0.2000
ROC 스코어: 0.5591, PR score : 0.0906


In [76]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 
same_x_train2 = same_x_train.copy()
same_x_test2 = same_x_test.copy()
same_y_train2 = same_y_train.copy()
same_y_test2 = same_y_test.copy()
same_x_train2.columns = [same_feature]
same_x_test2.columns =[same_feature]

xgb_wrapper = XGBClassifier()

xgb_wrapper.fit(same_x_train2, same_y_train2, verbose = True)


xgb_pred = xgb_wrapper.predict(same_x_test)
xgb_pred_proba = xgb_wrapper.predict_proba(same_x_test)

In [77]:
print("xgb") 
get_clf_eval(same_y_test, xgb_pred)

xgb
오차행렬
[[2301   14]
 [  40   10]]
정확도: 0.9772, 정밀도 : 0.4167, 재현율:0.2000,F1 스코어:0.2703
ROC 스코어: 0.5970, PR score : 0.1002


In [78]:
class Perceptron(object):
    
    def __init__(self, eta = 0.1, n_iter = 50, random_state = 1): #초기화
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
        
    
    def fit(self, X, y): #학습
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc = 0, scale = 0,
                             size = 1 + X.shape[1])
        
        self.errors_ = []
        for _ in range(self.n_iter):
            errors = 0
            for i in range(len(X)):
                xi = X.iloc[i].values
                target = y[i]
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def predict(self, X): #추론
        return np.where(self.net_input(X) >= 0.0, 1, 0)
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]


In [79]:
pc_clf = Perceptron(n_iter = 10)
pc_clf.fit(same_x_train, same_y_train)
pc_pred = pc_clf.predict(same_x_test)

In [80]:
print("퍼셉트론") 
get_clf_eval(same_y_test, pc_pred)

퍼셉트론
오차행렬
[[2306    9]
 [  45    5]]
정확도: 0.9772, 정밀도 : 0.3571, 재현율:0.1000,F1 스코어:0.1562
ROC 스코어: 0.5481, PR score : 0.0547
